In [36]:
import numpy as np
from tensorflow.keras.utils import to_categorical

raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

tokens = raw_text.split()
raw_text =' '.join(tokens)
char_vocab = sorted(list(set(raw_text)))
vocab_size = len(char_vocab)

char_to_index = dict((char, index) for index,char in enumerate(char_vocab) )
print(char_to_index)

length = 11
sequences = []
for i in range(length, len(raw_text)):
    seq = raw_text[i - length: i]
    sequences.append(seq)

sequences[:10]

encoded_sequences= []

for sequence in sequences:
    encoded_sequence = [char_to_index[char] for char in sequence]
    encoded_sequences.append(encoded_sequence)

encoded_sequences = np.array(encoded_sequences)
X_data = encoded_sequences[:, :-1]
y_data = encoded_sequences[:, -1]

X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)
print(X_data_one_hot.shape)





{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}
(426, 10, 33)


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

hidden_units = 64
model = Sequential()
model.add(LSTM(hidden_units, input_shape=(X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs=100, verbose=0)

In [58]:

def sentence_generation(model, char_to_index, seq_length, seed_text, n):
    init_text = seed_text
    sentence = ''
    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')
        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for char, index in char_to_index.items():
            if index == result:
                break
        
        seed_text = seed_text + char
        sentence = seed_text + char
    
    sentence = init_text + sentence
    return sentence
        

print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on wI get on with life as a programmer, I like to use words about beer. But when I stap my yaaa
